In [ ]:
!pip install selenium statsbombpy mplsoccer

In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch,Pitch
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns

In [3]:
free_comps = sb.competitions()
free_comps.head()

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-07-15T14:15:54.671676,2024-07-15T14:17:00.877356,2024-07-15T14:17:00.877356,2024-07-15T14:15:54.671676
1,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2024-05-19T11:11:14.192381,None,None,2024-05-19T11:11:14.192381
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-06-13T07:51:02.452825,None,None,2024-06-13T07:51:02.452825
3,16,4,Europe,Champions League,male,False,False,2018/2019,2024-06-12T07:44:38.559714,2021-06-13T16:17:31.694,None,2024-06-12T07:44:38.559714
4,16,1,Europe,Champions League,male,False,False,2017/2018,2024-02-13T02:35:28.134882,2021-06-13T16:17:31.694,None,2024-02-13T02:35:28.134882


In [4]:
can_2023_matches = sb.matches(competition_id=1267, season_id=107)

In [5]:
team = "Morocco"
matches_df = can_2023_matches[(can_2023_matches["home_team"] == team) | (can_2023_matches["away_team"] == team)]
matches_df = matches_df.sort_values(by="match_date",ascending=False)

In [6]:
match_ids = matches_df['match_id'].values
all_events = []
for match_id in match_ids:
    events_df = sb.events(match_id=match_id)
    all_events.append(events_df)
events_all_df = pd.concat(all_events, ignore_index=True)

In [7]:
events_all_df[["x","y"]] = events_all_df["location"].apply(pd.Series)
events_all_df[["pass_end_x","pass_end_y"]] = events_all_df["pass_end_location"].apply(pd.Series)
events_all_df[["carry_end_x","carry_end_y"]] = events_all_df["carry_end_location"].apply(pd.Series)

/tmp/ipython-input-7-1377057263.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  events_all_df[["carry_end_x","carry_end_y"]] = events_all_df["carry_end_location"].apply(pd.Series)


In [ ]:
f3rd_passes = events_all_df[(events_all_df.team == team) & (events_all_df.type == "Pass") & (events_all_df.x < 80 ) & (events_all_df.pass_end_x>80) & (events_all_df.pass_outcome.isna())]
f3rd_passes_count = f3rd_passes.groupby("player").size().reset_index()
f3rd_passes_count.rename(columns={f3rd_passes_count.columns[1]:"Passes"},inplace = True)
f3rd_carries = events_all_df[(events_all_df.type == "Carry") & (events_all_df.x < 80 ) & (events_all_df.carry_end_x>80) &(events_all_df.team == team) ]
f3rd_carries_count = f3rd_carries.groupby("player").size().reset_index()
f3rd_carries_count.rename(columns={f3rd_carries_count.columns[1]:"Carries"},inplace = True)
progression_df = pd.merge(f3rd_passes_count, f3rd_carries_count, how="outer",on=["player"])
progression_df = progression_df.fillna(0)
progression_df["total"] = progression_df["Passes"] + progression_df["Carries"]
progression_df.sort_values(by = "total",ascending = False,inplace=True)
# Bar Plot Showing Total Passes and Dribbles
plt.figure(figsize=(14, 10))
# Sort the dataframe by the 'total' column in descending order
progression_df_sorted = progression_df.sort_values(by='total', ascending=False)
# Create the bar plot
sns.barplot(x='total', y='player', data=progression_df_sorted, palette='viridis')
# Add titles and labels in English
plt.title(f"CAN 2023 - {team} - Players Moving the Ball to the Final Third (Passes + Dribbles)", fontsize=16)
plt.xlabel('Total Passes and Dribbles', fontsize=14)
plt.ylabel('Players', fontsize=14)
plt.savefig(f'/content/Output/CAN2023 - {team} - Players Moving the Ball to the Final Third (Passes + Dribbles).png')
# Display the plot
plt.show()

In [ ]:
# Pass Counts
plt.figure(figsize=(14, 10))
progression_df_sorted_passes = progression_df.sort_values(by='Passes', ascending=False)
sns.barplot(x='Passes', y='player', data=progression_df_sorted_passes, palette='Blues_r')
plt.title('Pass Counts Moving the Ball to the Final Third', fontsize=16)
plt.xlabel('Pass Count', fontsize=14)
plt.ylabel('Players', fontsize=14)
plt.savefig(f'/content/Output/CAN2023 - {team} - Pass Counts Moving the Ball to the Final Third.png')
plt.show()

# Carry Counts
plt.figure(figsize=(14, 10))

progression_df_sorted_carries = progression_df.sort_values(by='Carries', ascending=False)

sns.barplot(x='Carries', y='player', data=progression_df_sorted_carries, palette='Greens_r')
plt.title('Carry Counts Moving the Ball to the Final Third', fontsize=16)
plt.xlabel('Carry Count', fontsize=14)
plt.ylabel('Players', fontsize=14)
plt.savefig(f'/content/Output/CAN2023 -  {team}- Carry Counts Moving the Ball to the Final Third.png')
plt.show()

In [12]:
players = ['Sofyan Amrabat', 'Nayef Aguerd', 'Romain Saïss',
       'Noussair Mazraoui', 'Achraf Hakimi Mouh', 'Azzedine Ounahi',
       'Selim Amallah', 'Amine Adli', 'Yassine Bounou',
       'Abdessamad Ezzalzouli', 'Youssef En-Nesyri', 'Ismael Saibari',
       'Amine Harit', 'Yahia Attiyat allah', 'Ayoub El Kaabi',
       'Yunis Abdelhamid', 'Sofiane Boufal', 'Hakim Ziyech',
       'Amir Richardson', 'Tarik Tissoudali', 'Bilal El Khannous',
       'Mohamed Chibi']

In [ ]:
pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white')

for player in players:
    # Filtrer les passes réussies du joueur
    player_passes = events_all_df[
        (events_all_df.player == player) &
        (events_all_df.type == "Pass") &
        (events_all_df.pass_outcome.isna())
    ]

    fig, ax = plt.subplots(figsize=(8, 6))
    pitch.draw(ax=ax)

    # Tracer les points de départ et d'arrivée des passes
    for _, row in player_passes.iterrows():
        ax.plot(
            [row['x'], row['pass_end_x']],
            [row['y'], row['pass_end_y']],
            color='orange', linestyle='-', linewidth=2, alpha=0.5
        )

    # Marquer les points d'arrivée des passes
    if not player_passes.empty:
        pitch.scatter(
            x=player_passes.pass_end_x,
            y=player_passes.pass_end_y,
            c='red',
            label="Points d'arrivée des passes réussies",
            ax=ax,
            s=50,
            edgecolor='black'
        )

    ax.set_title(f'Passes Réussies de {player}', fontsize=14, fontweight='bold')
    ax.legend()
    plt.tight_layout()
    plt.savefig(f'/content/PassesRéussies/CAN 2024 - {team} - Passes Réussies - {player}.png')
    plt.show()


In [ ]:
pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white')

# Préparer les handles pour la légende
line_handle = plt.Line2D([], [], color='orange', linestyle='-', linewidth=2, label='Trajectoire des passes')
end_point_handle = plt.Line2D([], [], color='red', marker='o', linestyle='', markersize=10, label='Point de fin de passe', markeredgecolor='black')

# Un graphique séparé pour chaque joueur
for player in players:
    # Filtrer les passes incomplètes du joueur
    player_passes = events_all_df[
        (events_all_df.player == player) &
        (events_all_df.type == "Pass") &
        (events_all_df.pass_outcome == 'Incomplete')
    ]

    fig, ax = plt.subplots(figsize=(8, 6))
    pitch.draw(ax=ax)

    # Tracer les points de départ et d'arrivée des passes incomplètes
    for _, row in player_passes.iterrows():
        ax.plot(
            [row['x'], row['pass_end_x']],
            [row['y'], row['pass_end_y']],
            color='orange', linestyle='-', linewidth=2, alpha=0.5
        )

    # Marquer les points d'arrivée des passes incomplètes
    if not player_passes.empty:
        ax.scatter(
            x=player_passes.pass_end_x,
            y=player_passes.pass_end_y,
            color='red',
            s=70,
            edgecolor='black',
            marker='o'
        )

    # Ajouter le titre du graphique
    ax.set_title(f'Passes incomplètes de {player}', fontsize=14, fontweight='bold')

    # Légende spécifique à chaque graphique
    ax.legend(handles=[line_handle, end_point_handle], fontsize=10, loc='upper left')

    plt.tight_layout()
    plt.savefig(f'/content/Passes_incomplete/CAN2023 - {team} - Passes incomplètes - {player}.png')
    plt.show()

In [ ]:
# Liste des types d'actions
touches = ["Pass", "Ball Receipt*", "Carry", "Clearance", "Foul Won",
           "Block", "Ball Recovery", "Duel", "Dribble", "Interception",
           "Miscontrol", "Shot"]

# Définir la palette de couleurs
colour1 = "white"
colour2 = "#c3c3c3"
colour3 = "#e21017"
cmaplist = [colour1, colour2, colour3]
cmap = LinearSegmentedColormap.from_list("", cmaplist)

pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white')

# Stocker les couleurs pour chaque type d'événement, pour légende cohérente
event_colors = [cmap(0.3 + (i / len(touches))) for i in range(len(touches))]

# Handles pour la légende globale (un par type d'action)
legend_handles = [
    plt.Line2D([], [], color=event_colors[i], marker='o', linestyle='', markersize=10, label=touches[i], markeredgecolor='black')
    for i in range(len(touches))
]

# Un graphique séparé pour chaque joueur
for player in players:
    # Filtrer les événements du joueur
    player_df = events_all_df[
        (events_all_df['player'] == player) &
        (events_all_df['type'].isin(touches))
    ]

    fig, ax = plt.subplots(figsize=(8, 6))
    pitch.draw(ax=ax)

    # Tracer les événements pour chaque type
    for i, event_type in enumerate(touches):
        event_df = player_df[player_df['type'] == event_type]
        ax.scatter(
            x=event_df['x'],
            y=event_df['y'],
            color=event_colors[i],
            s=70,
            edgecolor='black',
            label=event_type  # Pour légende individuelle (optionnel)
        )

    # Ajouter le titre du graphique
    ax.set_title(f'Événements de {player}', fontsize=14, fontweight='bold')

    # Légende spécifique à chaque graphique
    ax.legend(handles=legend_handles, fontsize=10, loc='upper left', title="Types d'événements")

    plt.tight_layout()
    plt.savefig(f'/content/eventplayers/CAN2023 - {team} - Événements - {player}.png')
    plt.show()


In [ ]:
# Définir la palette de couleurs
colour1 = "white"
colour2 = "#c3c3c3"
colour3 = "#e21017"
cmaplist = [colour1, colour2, colour3]
cmap = LinearSegmentedColormap.from_list("", cmaplist)

# Path effect pour les labels
path_eff = [path_effects.Stroke(linewidth=2, foreground="black"), path_effects.Normal()]

# Saha (terrain) vertical
pitch = VerticalPitch(pitch_type="statsbomb", line_zorder=2, line_color="#000000", linewidth=2, half=False)

# Liste des joueurs
# Liste des types d'actions
touches = ["Pass", "Ball Receipt*", "Carry", "Clearance", "Foul Won",
           "Block", "Ball Recovery", "Duel", "Dribble", "Interception",
           "Miscontrol", "Shot"]

# Un graphique séparé pour chaque joueur
for player in players:
    player_df = events_all_df[(events_all_df.player == player) & (events_all_df.type.isin(touches))]
    fig, ax = plt.subplots(figsize=(8, 12), constrained_layout=True)

    # Calcul du bin_statistic (densité par zone du terrain)
    bin_statistic = pitch.bin_statistic(player_df.x, player_df.y, statistic="count", bins=(6, 4), normalize=True)
    vmax = bin_statistic["statistic"].max()
    vmin = 0

    # Affichage de la heatmap
    pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, vmax=vmax, vmin=vmin)

    # Affichage des pourcentages par zone
    pitch.label_heatmap(bin_statistic, color="white", path_effects=path_eff, fontsize=20, ax=ax,
                        str_format="{:.0%}", ha="center", va="center", exclude_zeros=True)

    ax.set_title(f"Activité de {player}", fontsize=18, loc='center', pad=0.4)

    pitch.draw(ax=ax)

    plt.tight_layout()
    plt.savefig(f'/content/heatmapactiv/CAN 2023 - {team} - Événements - {player}.png')
    plt.show()

In [ ]:
colour1 = "white"
colour2 = "#c3c3c3"
colour3 = "#e21017"
cmaplist = [colour1, colour2, colour3]
cmap = LinearSegmentedColormap.from_list("", cmaplist)

# Path effect pour les labels
path_eff = [path_effects.Stroke(linewidth=2, foreground="black"), path_effects.Normal()]

# Terrain vertical
pitch = VerticalPitch(pitch_type="statsbomb", line_zorder=2, line_color="#000000", linewidth=2, half=False)

# Liste des joueurs

# Liste des types d'actions
touches = ["Pass", "Ball Receipt*", "Carry", "Clearance", "Foul Won",
           "Block", "Ball Recovery", "Duel", "Dribble", "Interception",
           "Miscontrol", "Shot"]

# Un graphique séparé pour chaque joueur et chaque type d'action
for player in players:
    for event_type in touches:
        event_df = events_all_df[
            (events_all_df.player == player) &
            (events_all_df.type == event_type)
        ]
        if event_df.empty:
            continue  # On saute si pas d'événements pour ce joueur/type

        fig, ax = plt.subplots(figsize=(8, 12), constrained_layout=True)

        # Calcul du bin_statistic (densité par zone du terrain)
        bin_statistic = pitch.bin_statistic(event_df.x, event_df.y, statistic="count", bins=(6, 4), normalize=True)
        vmax = bin_statistic["statistic"].max()
        vmin = 0

        # Affichage de la heatmap
        pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, vmax=vmax, vmin=vmin)

        # Affichage des pourcentages par zone
        pitch.label_heatmap(bin_statistic, color="white", path_effects=path_eff, fontsize=20, ax=ax,
                            str_format="{:.0%}", ha="center", va="center", exclude_zeros=True)

        ax.set_title(f"{event_type} de {player}", fontsize=18, loc='center', pad=0.4)

        pitch.draw(ax=ax)

        plt.tight_layout()
        plt.savefig(f'/content/alleventperplayer/CAN 2023 - {team} - Événements - {player} - {event_type}.png')
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.colors as mcolors
import pandas as pd
# List of touch types to plot
touches = [
    "Pass", "Ball Receipt*", "Carry", "Clearance", "Foul Won",
    "Block", "Ball Recovery", "Duel", "Dribble", "Interception",
    "Miscontrol", "Shot"
]

# Custom colormap
colour1 = "white"
colour2 = "#c3c3c3"
colour3 = "#e21017"
cmaplist = [colour1, colour2, colour3]
cmap = mcolors.LinearSegmentedColormap.from_list("", cmaplist)

# Path effect settings
path_eff = [path_effects.Stroke(linewidth=2, foreground="black"), path_effects.Normal()]

# Pitch object setup (replace VerticalPitch with the correct import from mplsoccer or your own class)
from mplsoccer import VerticalPitch
pitch = VerticalPitch(pitch_type="statsbomb", line_zorder=2, line_color="#000000", linewidth=2, half=False)

# Loop over all touch types
for touch_type in touches:
    # Filter the events for the current touch type
    all_touches_df = pd.concat([
        events_all_df[(events_all_df['player'] == player) & (events_all_df['type'] == touch_type)]
        for player in players
    ])
    # Remove rows with NaN in coordinates
    all_touches_df = all_touches_df.dropna(subset=['x', 'y'])

    # Compute bin statistics only if dataframe is not empty
    if not all_touches_df.empty:
        bin_statistic = pitch.bin_statistic(
            all_touches_df['x'], all_touches_df['y'],
            statistic="count", bins=(6, 4), normalize=True
        )
        vmax = bin_statistic["statistic"].max()
        vmin = 0

        # Create the figure and axis
        fig, ax = plt.subplots(figsize=(15, 7))
        pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, vmax=vmax, vmin=vmin)
        pitch.label_heatmap(bin_statistic, color="white", path_effects=path_eff, fontsize=25, ax=ax,
                            str_format="{:.0%}", ha="center", va="center", exclude_zeros=True)
        pitch.draw(ax=ax)
        ax.set_facecolor('white')
        for spine in ax.spines.values():
            spine.set_color('black')
            spine.set_linewidth(6)
        ax.set_xticks([])
        ax.set_yticks([])
        fig.suptitle(f'{team} CAN 2023 : Carte thermique des {touch_type.lower()}', fontsize=24, fontweight='bold', y=1.02)
        plt.savefig(f'/content/Output/CAN 2023 -{team}- Événements - {touch_type}.png')
        plt.tight_layout()
        plt.show()
    else:
        print(f"Aucune donnée pour le type : {touch_type}")

In [20]:
matches_df

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
12,3922243,2024-01-30,22:00:00.000,Africa - African Cup of Nations,2023,Morocco,South Africa,0,2,available,...,None,4,Round of 16,Stade Laurent Pokou,Mahmood Ali Mahmood Ismail,Walid Regragui,Hugo Henri Broos,1.1.0,2,2
20,3920419,2024-01-24,22:00:00.000,Africa - African Cup of Nations,2023,Zambia,Morocco,0,1,available,...,None,3,Group Stage,Stade Laurent Pokou,Patrice Tanguy Mebiame,Avraham Grant,Walid Regragui,1.1.0,2,2
34,3920405,2024-01-21,16:00:00.000,Africa - African Cup of Nations,2023,Morocco,Congo DR,1,1,available,...,None,2,Group Stage,Stade Laurent Pokou,Peter Waweru Kamaku,Walid Regragui,Sébastien Desabre,1.1.0,2,2
44,3920394,2024-01-17,19:00:00.000,Africa - African Cup of Nations,2023,Morocco,Tanzania,3,0,available,...,None,1,Group Stage,Stade Laurent Pokou,Alhadi Allaou Mahamat,Walid Regragui,Adel Amrouche,1.1.0,2,2


In [ ]:
from mplsoccer import Pitch, Sbopen


team = "Morocco"


matches = {
     "Congo DR" : 3920405 ,
     "Tanzania": 3920394	,
     "South Africa" : 3922243	 ,
     "Zambia" : 3920419
}
for opponent, match_id in matches.items():
    parser = Sbopen()
    df, related, freeze, tactics = parser.event(match_id)


    df = df[df['team_name'] == team]


    passes = df[df['type_name'] == 'Pass'].copy()
    passes['outcome_name'] = passes['outcome_name'].fillna('Successful')


    plt.figure(figsize=(10, 6))
    pass_outcome_counts = passes['outcome_name'].value_counts().sort_values()
    pass_outcome_counts.plot(kind='barh', color='red')
    plt.xlabel('Number of Passes')
    plt.ylabel('Pass Outcome')
    plt.title(f'Pass Outcomes by {team} Against {opponent} in CAN 2023')
    plt.savefig(f"/content/Output/{team}'s CAN 2023 Pass Outcomes Against {opponent}")
    plt.show()


    successful_passes = passes[passes['outcome_name'] == 'Successful']


    first_sub = df[df['type_name'] == 'Substitution']['minute'].min()
    successful_passes = successful_passes[successful_passes['minute'] < first_sub]


    passer_avg_loc = successful_passes.groupby('player_name').agg(
        avg_x=('x', 'mean'),
        avg_y=('y', 'mean'),
        pass_count=('id', 'size')
    ).reset_index()


    pass_between = successful_passes.groupby(['player_name', 'pass_recipient_name']).size().reset_index(name='pass_between_count')


    pass_between = pass_between.merge(passer_avg_loc, left_on='player_name', right_on='player_name')
    pass_between = pass_between.merge(passer_avg_loc, left_on='pass_recipient_name', right_on='player_name', suffixes=['', '_end'])


    passer_avg_loc['player_name'] = passer_avg_loc['player_name'].apply(lambda name: name.split()[-1])
    pass_between['player_name'] = pass_between['player_name'].apply(lambda name: name.split()[-1])
    pass_between['pass_recipient_name'] = pass_between['pass_recipient_name'].apply(lambda name: name.split()[-1])


    pitch_length_x = 120
    pitch_width_y = 80
    pitch = Pitch(pitch_type='custom', pitch_length=pitch_length_x, pitch_width=pitch_width_y, line_color='black')
    fig, ax = pitch.draw(figsize=(10, 7))


    pitch.lines(
        1.2 * pass_between.avg_x,
        0.8 * pass_between.avg_y,
        1.2 * pass_between.avg_x_end,
        0.8 * pass_between.avg_y_end,
        lw=pass_between.pass_between_count * 0.5,
        color='red',
        zorder=1,
        ax=ax
    )


    pitch.scatter(
        1.2 * passer_avg_loc.avg_x,
        0.8 * passer_avg_loc.avg_y,
        s=20 * passer_avg_loc['pass_count'].values,
        color='white',
        edgecolors='red',
        linewidth=2,
        alpha=1,
        zorder=1,
        ax=ax
    )


    for index, row in passer_avg_loc.iterrows():
        pitch.annotate(
            row['player_name'],
            xy=(1.2 * row.avg_x, 0.8 * row.avg_y),
            c='black',
            fontweight='bold',
            va='center',
            ha='center',
            size=8,
            ax=ax
        )


    fig.suptitle(f"{team}'s CAN 2023 Pass Networks Against {opponent}", fontsize=16)
    ax.text(
        0.95, -0.05,
        f"First substitution occurred at {first_sub}'",
        color='gray',
        va='bottom',
        ha='right',
        fontsize=10,
        fontstyle='italic',
        transform=ax.transAxes
    )
    plt.savefig(f"/content/Output/{team}'s CAN 2023 Pass Networks Against {opponent}")
    plt.show()

In [22]:
import shutil



shutil.make_archive('/content/resultat/Match_Output', 'zip', "/content/Output")
shutil.make_archive('/content/resultat/Successful_Passes', 'zip', "/content/PassesRéussies")
shutil.make_archive('/content/resultat/Incomplete_Passes', 'zip', "/content/Passes_incomplete")
shutil.make_archive('/content/resultat/Activity_Heatmap', 'zip', "/content/heatmapactiv")
shutil.make_archive('/content/resultat/Player_Events', 'zip', "/content/eventplayers")
shutil.make_archive('/content/resultat/All_Events_Per_Player', 'zip', "/content/alleventperplayer")
shutil.make_archive("resultat_Morocco", 'zip', "/content/resultat")

'/content/resultat_Morocco.zip'